In [1]:
#QUESTION1
class MedicalDiagnosisAgent:
   #instance
    
    def __init__(self):
        #dictionary self.diseases that stores diseases 
        
        self.diseases = {
            "Acute Appendicitis": {
                "symptoms": ["fever", "pain in abdomen", "vomiting"],
                "tests": {
                    "blood cp with esr": {
                        "tlc": "high",
                        "dlc neutrophils": "high",
                        "esr": "high"
                    }
                },
                "treatment": "Surgery"
            },
            "Pneumonia": {
                "symptoms": ["fever", "cough", "chest pain"],
                "tests": {
                    "blood cp with esr": {
                        "tlc": "high",
                        "dlc neutrophils": "high",
                        "esr": "high"
                    },
                    "x-ray chest": "pneumonic patch"
                },
                "treatment": "Antibiotics"
            },
            "Acute Tonsillitis": {
                "symptoms": ["fever", "cough"],
                "tests": {
                    "examine throat": {
                        "red enlarged tonsils": True,
                        "pus in tonsils": True
                    }
                },
                "treatment": "Anti-allergic + Paracetamol. If not cured: Add antibiotics orally. If still not cured: Add IV antibiotics."
            }
        }
    
    def get_user_symptoms(self):
        #taking symptoms from user
        print("\n=== MEDICAL DIAGNOSIS SYSTEM ===")
        print("Please enter your symptoms below (comma-separated):")
        symptoms_input = input("> ")
        
       
        symptoms = [s.strip().lower() for s in symptoms_input.split(",")]
        return symptoms
    
    def get_test_results(self):
        #taking test results from user
        print("\nPlease enter any test results (comma-separated):")
        print("Example: TLC high, DLC Neutrophils high, X-ray shows pneumonic patch")
        tests_input = input("> ")
        
        
        test_results = {}
        if tests_input.strip():
            test_pairs = [t.strip().lower() for t in tests_input.split(",")]
            
            for pair in test_pairs:
                if ":" in pair:
                    key, value = pair.split(":", 1)
                elif " shows " in pair:
                    key, value = pair.split(" shows ", 1)
                elif " show " in pair:
                    key, value = pair.split(" show ", 1)
                else:
                    parts = pair.split()
                    if len(parts) >= 2:
                        key = " ".join(parts[:-1])
                        value = parts[-1]
                    else:
                        key = pair
                        value = "positive"  
                
                test_results[key.strip()] = value.strip()
        
        return test_results
    
    def diagnose(self, symptoms, test_results):
        
        potential_diseases = []
        
        #Comparing User Symptoms with Each Disease
        for disease, details in self.diseases.items():
            
            required_symptoms = details["symptoms"]
            matching_symptoms = sum(1 for s in required_symptoms if any(s in user_s for user_s in symptoms))
            
            
            symptom_ratio = matching_symptoms / len(required_symptoms)
            if symptom_ratio >= 0.75:  
                #If 75% or more symptoms match, the disease is added to potential_diseases.
                potential_diseases.append((disease, symptom_ratio))
        
        
        potential_diseases.sort(key=lambda x: x[1], reverse=True)
        
        #if no disease matches
        if not potential_diseases:
            return "Unknown", "Please consult with a healthcare professional for further evaluation."
        
        
        confirmed_disease = None
        for disease, _ in potential_diseases:
            disease_details = self.diseases[disease]
            required_tests = disease_details.get("tests", {})
            
            
            if not required_tests:
                confirmed_disease = disease
                break
            
            
            test_match = self._verify_test_results(required_tests, test_results)
            if test_match:
                confirmed_disease = disease
                break
        
        
        if not confirmed_disease and potential_diseases:
            confirmed_disease = potential_diseases[0][0]  
            
        
        if confirmed_disease:
            treatment = self.diseases[confirmed_disease]["treatment"]
            return confirmed_disease, treatment
        else:
            return "Unknown", "Please consult with a healthcare professional for further evaluation."
    
    def _verify_test_results(self, required_tests, user_tests):
        
        
        for test_type, required_results in required_tests.items():
            if isinstance(required_results, dict):
                
                for param, required_value in required_results.items():
                    
                    param_matched = False
                    for user_test, user_value in user_tests.items():
                        if param in user_test and required_value in user_value:
                            param_matched = True
                            break
                    
                    if not param_matched:
                        return False 
            else:
                
                test_matched = False
                for user_test, user_value in user_tests.items():
                    if test_type in user_test and required_results in user_value:
                        test_matched = True
                        break
                
                if not test_matched:
                    return False 
        
        return True  
    
    def run(self):
       
        symptoms = self.get_user_symptoms()
        test_results = self.get_test_results()
        
        diagnosis, treatment = self.diagnose(symptoms, test_results)
        
        print("\n=== DIAGNOSIS RESULTS ===")
        print(f"Diagnosed Disease: {diagnosis}")
        print(f"Recommended Treatment: {treatment}")
        
        


if __name__ == "__main__":
    agent = MedicalDiagnosisAgent()
    agent.run()


=== MEDICAL DIAGNOSIS SYSTEM ===
Please enter your symptoms below (comma-separated):

Please enter any test results (comma-separated):
Example: TLC high, DLC Neutrophils high, X-ray shows pneumonic patch

=== DIAGNOSIS RESULTS ===
Diagnosed Disease: Acute Tonsillitis
Recommended Treatment: Anti-allergic + Paracetamol. If not cured: Add antibiotics orally. If still not cured: Add IV antibiotics.


In [ ]:
#Model based agent
#QUESTION2
import random
import time
import copy

class WumpusWorld:
    def __init__(self, grid):
        
        self.grid = grid
        self.rows = len(grid)
        self.cols = len(grid[0]) if self.rows > 0 else 0
        
    def get_percept(self, position):
       
        row, col = position
        if 0 <= row < self.rows and 0 <= col < self.cols:
            return self.grid[row][col]
        return None
    
    def get_adjacent_percepts(self, position):
        
        row, col = position
        adjacent_positions = {
            "up": (row - 1, col),
            "down": (row + 1, col),
            "left": (row, col - 1),
            "right": (row, col + 1)
        }
        
        # Randomize the order of percepts
        directions = list(adjacent_positions.keys())
        random.shuffle(directions)
        
        percepts = {}
        for direction in directions:
            adj_pos = adjacent_positions[direction]
            if 0 <= adj_pos[0] < self.rows and 0 <= adj_pos[1] < self.cols:
                percepts[direction] = self.grid[adj_pos[0]][adj_pos[1]]
            else:
                percepts[direction] = "Wall"  # Out of bounds
        
        return percepts


class Agent:
    def __init__(self, world):
        
        self.world = world
        self.position = (0, 0)  # Start at top-left corner
        self.has_gold = False
        self.internal_model = [["Unknown" for _ in range(world.cols)] for _ in range(world.rows)]
        self.internal_model[0][0] = "Start"  # Mark starting position
        self.visited = set([(0, 0)])  # Track visited positions
        self.path = [(0, 0)]  # Track the path taken
        
    def update_model(self, position, percept):
        
        row, col = position
        if 0 <= row < self.world.rows and 0 <= col < self.world.cols:
            self.internal_model[row][col] = percept
    
    def assess_safety(self, position):
        
        row, col = position
        
        # Checking if position is valid
        if not (0 <= row < self.world.rows and 0 <= col < self.world.cols):
            return -float('inf')  # Invalid position
        
        percept = self.internal_model[row][col]
        
        
        if percept in ["Pit", "Wumpus"]:
            return -float('inf')
        
        
        if percept in ["Empty", "Gold"]:
            return 2.0 if position not in self.visited else 1.0
        
        
        if percept == "Unknown":
            return 0.5
        
        # Default case
        return 0.0
    
    def get_best_action(self):
        
        # If we're on gold, grab it
        current_percept = self.world.get_percept(self.position)
        if current_percept == "Gold" and not self.has_gold:
            return "Grab gold"
        
        # If we have gold, head back to start
        if self.has_gold:
            return self.get_return_action()
        
        # Otherwise, explore to find gold
        return self.get_exploration_action()
    
    def get_exploration_action(self):
        
        adjacent_positions = {
            "up": (self.position[0] - 1, self.position[1]),
            "down": (self.position[0] + 1, self.position[1]),
            "left": (self.position[0], self.position[1] - 1),
            "right": (self.position[0], self.position[1] + 1)
        }
        
        
        adjacent_percepts = self.world.get_adjacent_percepts(self.position)
        for direction, percept in adjacent_percepts.items():
            if direction in adjacent_positions:
                self.update_model(adjacent_positions[direction], percept)
        
        
        safety_scores = {}
        for direction, pos in adjacent_positions.items():
            safety_scores[direction] = self.assess_safety(pos)
        
        #  safest direction
        best_direction = max(safety_scores, key=safety_scores.get)
        best_score = safety_scores[best_direction]
        
        # If all directions are unsafe, doing nothing
        if best_score <= -float('inf'):
            return "Do nothing"
        
        # best direction
        return f"Move {best_direction}"
    
    def get_return_action(self):
        #best action to return to start

        # If already at start, we're done
        if self.position == (0, 0):
            return "Do nothing"
        
        # Using A* pathfinding to find the safest path back
        path = self.find_path_to_start()
        if not path:
            # If no safe path found, try to find any path using existing knowledge
            return self.get_exploration_action()
        
        
        next_pos = path[1]  # path[0] is current position
        row_diff = next_pos[0] - self.position[0]
        col_diff = next_pos[1] - self.position[1]
        
        if row_diff == -1:
            return "Move up"
        elif row_diff == 1:
            return "Move down"
        elif col_diff == -1:
            return "Move left"
        elif col_diff == 1:
            return "Move right"
        
        
        return "Do nothing"
    
    def find_path_to_start(self):
        
        from heapq import heappush, heappop
        
        start = self.position
        goal = (0, 0)
        
        
        if start == goal:
            return [start]
        
        # A* algorithm
        open_set = []
        heappush(open_set, (0, start))
        came_from = {}
        g_score = {start: 0}
        f_score = {start: self.heuristic(start, goal)}
        
        open_set_hash = {start}
        
        while open_set:
            _, current = heappop(open_set)
            open_set_hash.remove(current)
            
            if current == goal:
                
                path = [current]
                while current in came_from:
                    current = came_from[current]
                    path.append(current)
                path.reverse()
                return path
            
            # Checking all adjacent positions
            for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
                neighbor = (current[0] + dx, current[1] + dy)
                
                # Checking if neighbor is valid and safe
                if (0 <= neighbor[0] < self.world.rows and 
                    0 <= neighbor[1] < self.world.cols and
                    self.assess_safety(neighbor) > -float('inf')):
                    
                    tentative_g_score = g_score[current] + 1
                    
                    if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                        # better path
                        came_from[neighbor] = current
                        g_score[neighbor] = tentative_g_score
                        f_score[neighbor] = tentative_g_score + self.heuristic(neighbor, goal)
                        
                        if neighbor not in open_set_hash:
                            heappush(open_set, (f_score[neighbor], neighbor))
                            open_set_hash.add(neighbor)
        
        # No path found
        return None
    
    def heuristic(self, a, b):
        #manhatan distance
        return abs(a[0] - b[0]) + abs(a[1] - b[1])
    
    def execute_action(self, action):
        
        if action == "Do nothing":
            return True
        
        elif action == "Grab gold":
            current_percept = self.world.get_percept(self.position)
            if current_percept == "Gold":
                self.has_gold = True
                # internal model to show we've taken the gold
                self.update_model(self.position, "Empty")
                return True
            return False
        
        elif action.startswith("Move"):
            direction = action.split(" ")[1]
            new_position = self.position
            
            if direction == "up" and self.position[0] > 0:
                new_position = (self.position[0] - 1, self.position[1])
            elif direction == "down" and self.position[0] < self.world.rows - 1:
                new_position = (self.position[0] + 1, self.position[1])
            elif direction == "left" and self.position[1] > 0:
                new_position = (self.position[0], self.position[1] - 1)
            elif direction == "right" and self.position[1] < self.world.cols - 1:
                new_position = (self.position[0], self.position[1] + 1)
            
            # Checking if new position is valid
            percept = self.world.get_percept(new_position)
            if percept is not None:
                self.position = new_position
                self.visited.add(new_position)
                self.path.append(new_position)
                self.update_model(new_position, percept)
                
                
                if percept in ["Pit", "Wumpus"]:
                    return False  # Agent dies
                
                return True
        
        return False
    
    def print_state(self):
        #printing current state of agent and world
        print(f"Current Position: {self.position}")
        print("Has Gold: ", self.has_gold)
        print("State of the World (Internal Model):")
        
        # Creating a copy of the internal model for display
        display_model = copy.deepcopy(self.internal_model)
        display_model[self.position[0]][self.position[1]] += " (Agent)"
        
        print("[")
        for row in display_model:
            print(f"  {row}")
        print("]")
    
    def run(self, max_steps=50):
        
        steps = 0
        success = False
        
        while steps < max_steps:
            self.print_state()
            
            action = self.get_best_action()
            print(f"Next Action: {action}")
            
            result = self.execute_action(action)
            
            # Checking termination conditions
            if not result:
                print("Agent failed! (Fell into a pit or encountered the Wumpus)")
                return False
            
            if action == "Do nothing" and self.has_gold and self.position == (0, 0):
                success = True
                break
            
            if action == "Do nothing" and not self.has_gold:
                print("Agent is stuck with no safe moves.")
                return False
            
            steps += 1
            input("Press Enter to continue...")
            print("\n" + "-"*50 + "\n")
        
        if steps >= max_steps:
            print(f"Agent exceeded maximum steps ({max_steps}).")
            return False
        
        if success:
            print("Success! Agent found the gold and returned to start.")
            return True
        
        return False


def main():
    #  Wumpus World from the problem statement
    grid = [
        ["Start", "Empty", "Empty", "Pit"],
        ["Empty", "Pit", "Wumpus", "Empty"],
        ["Empty", "Gold", "Empty", "Pit"],
        ["Empty", "Pit", "Empty", "Empty"]
    ]
    
    # Creating the world
    world = WumpusWorld(grid)
    
    # Creating the agent
    agent = Agent(world)
    
    # Run the agent
    print("=== Wumpus World Agent ===")
    print("Starting agent...")
    result = agent.run()
    
    if result:
        print("Mission accomplished! The agent found the gold and returned safely.")
    else:
        print("Mission failed! The agent could not complete the task.")


if __name__ == "__main__":
    main()

=== Wumpus World Agent ===
Starting agent...
Current Position: (0, 0)
Has Gold:  False
State of the World (Internal Model):
[
  ['Start (Agent)', 'Unknown', 'Unknown', 'Unknown']
  ['Unknown', 'Unknown', 'Unknown', 'Unknown']
  ['Unknown', 'Unknown', 'Unknown', 'Unknown']
  ['Unknown', 'Unknown', 'Unknown', 'Unknown']
]
Next Action: Move down

--------------------------------------------------

Current Position: (1, 0)
Has Gold:  False
State of the World (Internal Model):
[
  ['Start', 'Empty', 'Unknown', 'Unknown']
  ['Empty (Agent)', 'Unknown', 'Unknown', 'Unknown']
  ['Unknown', 'Unknown', 'Unknown', 'Unknown']
  ['Unknown', 'Unknown', 'Unknown', 'Unknown']
]
Next Action: Move down

--------------------------------------------------

Current Position: (2, 0)
Has Gold:  False
State of the World (Internal Model):
[
  ['Start', 'Empty', 'Unknown', 'Unknown']
  ['Empty', 'Pit', 'Unknown', 'Unknown']
  ['Empty (Agent)', 'Unknown', 'Unknown', 'Unknown']
  ['Unknown', 'Unknown', 'Unknown',

In [ ]:
#Goal based agent
#QUESTION3
import numpy as np
import time

class PacmanGame:
    
    EMPTY = 0
    PACMAN = 1
    FOOD = 2
    POWER_PELLET = 3
    GHOST = 4
    WALL = 5

    def __init__(self):
        
        self.grid = np.zeros((4, 4), dtype=int)
        
        #  Pacman at position 0
        self.grid[0, 0] = self.PACMAN
        self.pacman_pos = (0, 0)
        
        #  power pellets (cherries)
        cherry_positions = [(0, 1), (0, 3), (2, 0), (2, 2), (2, 3), (3, 0)]
        for pos in cherry_positions:
            self.grid[pos] = self.POWER_PELLET
        
        #  food pellets
        food_positions = [(0, 2), (1, 0), (1, 2), (1, 3), (2, 1), (3, 1), (3, 3)]
        for pos in food_positions:
            self.grid[pos] = self.FOOD
        
        #  ghosts
        ghost_positions = [(1, 1), (2, 3), (3, 2)]
        for pos in ghost_positions:
            self.grid[pos] = self.GHOST
        
        
        self.walls = [
            ((0, 1), (0, 2)),  # Wall between cell 1 and 2
            ((1, 2), (1, 3)),  # Wall between cell 6 and 7
            ((2, 0), (2, 1)),  # Wall between cell 8 and 9
            ((2, 2), (2, 3)),  # Wall between cell 10 and 11
            ((3, 0), (3, 1)),  # Wall between cell 12 and 13
        ]
        
        # Game state
        self.power_mode = False
        self.power_time = 0
        self.score = 0
        self.game_over = False
        self.win = False
        self.moves = []
        
    def get_cell_name(self, i, j):
        return i * 4 + j
    
    def is_wall_between(self, pos1, pos2):
        """Check if there's a wall between two adjacent positions"""
        for wall_start, wall_end in self.walls:
            if (pos1 == wall_start and pos2 == wall_end) or (pos1 == wall_end and pos2 == wall_start):
                return True
        return False
    
    def get_valid_moves(self):
        """Get all valid moves from current position"""
        i, j = self.pacman_pos
        possible_moves = []
        
        # Check all four directions
        directions = [
            ('UP', (-1, 0)),
            ('DOWN', (1, 0)),
            ('LEFT', (0, -1)),
            ('RIGHT', (0, 1))
        ]
        
        for direction, (di, dj) in directions:
            new_i, new_j = i + di, j + dj
            
            # Checking if the new position is within the grid
            if 0 <= new_i < 4 and 0 <= new_j < 4:
                # Checking if there's no wall between current and new position
                if not self.is_wall_between((i, j), (new_i, new_j)):
                    possible_moves.append((direction, (new_i, new_j)))
        
        return possible_moves
    
    def calculate_utility(self, new_pos):
        """Calculate the utility value for moving to a new position"""
        i, j = new_pos
        cell_content = self.grid[i, j]
        
        # utility values
        utilities = {
            self.EMPTY: 0,
            self.FOOD: 10,
            self.POWER_PELLET: 20,
            self.GHOST: -100 if not self.power_mode else 15
        }
        
        # Calculating distance to nearest food or power pellet if current cell is empty
        if cell_content == self.EMPTY:
            min_distance = float('inf')
            for i_food in range(4):
                for j_food in range(4):
                    if self.grid[i_food, j_food] in [self.FOOD, self.POWER_PELLET]:
                        distance = abs(i - i_food) + abs(j - j_food)
                        min_distance = min(min_distance, distance)
            
            # If there are still food or power pellets, add distance-based utility
            if min_distance != float('inf'):
                return utilities[cell_content] + 5 / (min_distance + 1)
        
        return utilities.get(cell_content, 0)
    
    def make_move(self, direction, new_pos):
        """Execute a move in the given direction"""
        self.grid[self.pacman_pos] = self.EMPTY
        i, j = new_pos
        
        # Checking what's in the new cell
        cell_content = self.grid[i, j]
        
        # Updating game state based on cell content
        if cell_content == self.GHOST and not self.power_mode:
            self.game_over = True
            print(f"Game over! Pacman eaten by ghost at cell {self.get_cell_name(i, j)}")
            return False
        
        elif cell_content == self.POWER_PELLET:
            self.power_mode = True
            self.power_time = 3  # Power lasts for 3 moves
            self.score += 20
            print(f"Pacman ate a power pellet! Power mode activated for 3 moves.")
        
        elif cell_content == self.FOOD:
            self.score += 10
        
        # Moving Pacman to the new position
        self.grid[i, j] = self.PACMAN
        self.pacman_pos = (i, j)
        self.moves.append(direction)
        
        
        if self.power_mode:
            self.power_time -= 1
            if self.power_time <= 0:
                self.power_mode = False
                print("Power mode deactivated.")
        
        #  if all food and power pellets are consumed
        if not np.any((self.grid == self.FOOD) | (self.grid == self.POWER_PELLET)):
            self.win = True
            print("Victory! All food and power pellets have been consumed.")
            return False
        
        return True
    
    def select_best_move(self):
        """Select the best move based on utility"""
        valid_moves = self.get_valid_moves()
        best_move = None
        best_utility = float('-inf')
        
        for direction, new_pos in valid_moves:
            utility = self.calculate_utility(new_pos)
            if utility > best_utility:
                best_utility = utility
                best_move = (direction, new_pos)
        
        return best_move, best_utility
    
    def print_grid(self):
        """Print the current state of the grid"""
        symbols = {
            self.EMPTY: ".",
            self.PACMAN: "P",
            self.FOOD: "F",
            self.POWER_PELLET: "C",  
            self.GHOST: "G",
            self.WALL: "|"
        }
        
        print("\nCurrent Grid:")
        print("-" * 9)
        for i in range(4):
            row = ""
            for j in range(4):
                cell_num = i * 4 + j
                cell_sym = symbols[self.grid[i, j]]
                row += f"{cell_sym} "
            print(row)
        print("-" * 9)
        
        # Priningt additional information
        remaining_food = np.sum(self.grid == self.FOOD)
        remaining_power = np.sum(self.grid == self.POWER_PELLET)
        print(f"Pacman at cell: {self.get_cell_name(*self.pacman_pos)}")
        print(f"Power mode: {'ON' if self.power_mode else 'OFF'}")
        print(f"Remaining food pellets: {remaining_food}")
        print(f"Remaining power pellets: {remaining_power}")
        print(f"Score: {self.score}")
    
    def play(self):
        """Play the game using the utility-based agent"""
        print("Starting Pacman Game...")
        self.print_grid()
        
        move_count = 0
        while not self.game_over and not self.win:
            # Delay for readability
            time.sleep(0.5)
            
            # Selecting the best move
            (direction, new_pos), utility = self.select_best_move()
            
            # Printing the chosen move and its utility
            print(f"\nMove {move_count + 1}: {direction} (Utility: {utility:.2f})")
            
            # Executing the move
            continue_game = self.make_move(direction, new_pos)
            if not continue_game:
                break
            
            # Printing the updated grid
            self.print_grid()
            move_count += 1
            
            # Safety check for infinite loops
            if move_count > 50:
                print("Maximum number of moves reached. Ending game.")
                break
        
        # Printing final result
        print("\nGame Finished!")
        if self.win:
            print("Pacman won by collecting all pellets!")
        elif self.game_over:
            print("Pacman was caught by a ghost.")
        
        print(f"Final score: {self.score}")
        print(f"Moves made: {len(self.moves)}")
        print(f"Path taken: {' -> '.join(self.moves)}")

# Run the game
if __name__ == "__main__":
    game = PacmanGame()
    game.play()

Starting Pacman Game...

Current Grid:
---------
P C F C 
F G F F 
C F C G 
C F G F 
---------
Pacman at cell: 0
Power mode: OFF
Remaining food pellets: 7
Remaining power pellets: 5
Score: 0

Move 1: RIGHT (Utility: 20.00)
Pacman ate a power pellet! Power mode activated for 3 moves.

Current Grid:
---------
. P F C 
F G F F 
C F C G 
C F G F 
---------
Pacman at cell: 1
Power mode: ON
Remaining food pellets: 7
Remaining power pellets: 4
Score: 20

Move 2: DOWN (Utility: 15.00)

Current Grid:
---------
. . F C 
F P F F 
C F C G 
C F G F 
---------
Pacman at cell: 5
Power mode: ON
Remaining food pellets: 7
Remaining power pellets: 4
Score: 20

Move 3: DOWN (Utility: 10.00)
Power mode deactivated.

Current Grid:
---------
. . F C 
F . F F 
C P C G 
C F G F 
---------
Pacman at cell: 9
Power mode: OFF
Remaining food pellets: 6
Remaining power pellets: 4
Score: 30

Move 4: RIGHT (Utility: 20.00)
Pacman ate a power pellet! Power mode activated for 3 moves.

Current Grid:
---------
. . F C 
F

In [7]:
#QUESTION4
class RoomServiceRobot:
    def __init__(self):
        self.rooms = {'A', 'B', 'C'}
        self.service_room = 'Service Room'
        self.actions = {
            'A': {'action': 'Left move + Serve', 'cost': 5},
            'B': {'action': 'Right move + Serve', 'cost': 5},
            'C': {'action': 'Up move + Serve', 'cost': 1},
            'Back': {'action': 'Back to Service Room', 'cost': 0}
        }
        self.initial_state = {
            'robot_position': self.service_room,
            'served_rooms': set(),
            'total_cost': 0,
            'path': [self.service_room]
        }

    def is_goal_state(self, state):
        return state['served_rooms'] == self.rooms and state['robot_position'] == self.service_room

    def get_possible_actions(self, state):
        if state['robot_position'] == self.service_room:
            return [room for room in self.rooms if room not in state['served_rooms']]
        else:
            return ['Back']

    def apply_action(self, state, action):
        new_state = {
            'robot_position': self.service_room if action == 'Back' else action,
            'served_rooms': state['served_rooms'].copy(),
            'total_cost': state['total_cost'] + self.actions[action]['cost'],
            'path': state['path'] + [action]
        }
        if action != 'Back':
            new_state['served_rooms'].add(action)
        return new_state

    def dfs(self):
        stack = [self.initial_state]
        while stack:
            current_state = stack.pop()
            if self.is_goal_state(current_state):
                return current_state['path'], current_state['total_cost']
            for action in self.get_possible_actions(current_state):
                new_state = self.apply_action(current_state, action)
                stack.append(new_state)
        return None, float('inf')

    def iddfs(self):
        depth = 0
        while True:
            result = self.depth_limited_dfs(self.initial_state, depth)
            if result:
                return result
            depth += 1

    def depth_limited_dfs(self, state, depth):
        if depth == 0:
            if self.is_goal_state(state):
                return state['path'], state['total_cost']
            else:
                return None
        for action in self.get_possible_actions(state):
            new_state = self.apply_action(state, action)
            result = self.depth_limited_dfs(new_state, depth - 1)
            if result:
                return result
        return None


robot = RoomServiceRobot()
dfs_path, dfs_cost = robot.dfs()
iddfs_path, iddfs_cost = robot.iddfs()

print("DFS Path:", " → ".join(dfs_path))
print("DFS Total Cost:", dfs_cost)
print("IDDFS Path:", " → ".join(iddfs_path))
print("IDDFS Total Cost:", iddfs_cost)

DFS Path: Service Room → B → Back → C → Back → A → Back
DFS Total Cost: 11
IDDFS Path: Service Room → A → Back → C → Back → B → Back
IDDFS Total Cost: 11


In [11]:
#Bridge and torch problem using uniform cost search
#QUESTION5
import heapq

class State:
    def __init__(self, bank_a, bank_b, flashlight_at_a, time=0, moves=None):
      
        self.bank_a = frozenset(bank_a) #tourist at bank a
        self.bank_b = frozenset(bank_b) #tourist at bank b
        self.flashlight_at_a = flashlight_at_a
        self.time = time
        self.moves = moves if moves is not None else []
    
    def is_goal(self):
        
        return len(self.bank_a) == 0
    
    def __lt__(self, other):
        
        return self.time < other.time
    
    def __eq__(self, other):
        
        return (self.bank_a == other.bank_a and 
                self.bank_b == other.bank_b and 
                self.flashlight_at_a == other.flashlight_at_a)
    
    def __hash__(self):
        
        return hash((self.bank_a, self.bank_b, self.flashlight_at_a))


def get_successors(state, crossing_times):
   
    successors = []
    
    # If flashlight is at bank A, move 1 or 2 tourists from A to B
    if state.flashlight_at_a:
        # Move 1 tourist
        for tourist in state.bank_a:
            new_bank_a = set(state.bank_a) - {tourist}
            new_bank_b = set(state.bank_b) | {tourist}
            
            # Time taken is the crossing time of the tourist
            time_taken = crossing_times[tourist]
            new_time = state.time + time_taken
            
            # Create move description
            move = f"({tourist}) cross → Time: {new_time}"
            
            # Create new state
            new_moves = state.moves + [move]
            successor = State(new_bank_a, new_bank_b, False, new_time, new_moves)
            successors.append(successor)
        
        # Move 2 tourists
        tourists_list = sorted(list(state.bank_a))  # Sorting for consistent ordering
        for i, tourist1 in enumerate(tourists_list):
            for j, tourist2 in enumerate(tourists_list[i+1:], i+1):
                new_bank_a = set(state.bank_a) - {tourist1, tourist2}
                new_bank_b = set(state.bank_b) | {tourist1, tourist2}
                
                # Time taken is the maximum of the two tourists' crossing times
                time_taken = max(crossing_times[tourist1], crossing_times[tourist2])
                new_time = state.time + time_taken
                
                # Creating move description
                move = f"({tourist1},{tourist2}) cross → Time: {new_time}"
                
                # Creating new state
                new_moves = state.moves + [move]
                successor = State(new_bank_a, new_bank_b, False, new_time, new_moves)
                successors.append(successor)
    
    # If flashlight is at bank B, move 1 tourist back to A
    else:
        # Sorting tourists for consistent ordering when all have same crossing time
        tourists_list = sorted(list(state.bank_b))
        for tourist in tourists_list:
            new_bank_a = set(state.bank_a) | {tourist}
            new_bank_b = set(state.bank_b) - {tourist}
            
            # Time taken is the crossing time of the tourist
            time_taken = crossing_times[tourist]
            new_time = state.time + time_taken
            
            # Creatign move description
            move = f"({tourist}) returns → Time: {new_time}"
            
            # Creating new state
            new_moves = state.moves + [move]
            successor = State(new_bank_a, new_bank_b, True, new_time, new_moves)
            successors.append(successor)
    
    return successors


def uniform_cost_search(crossing_times):
    
    # Create initial state with all tourists on bank A
    tourists = set(crossing_times.keys())
    initial_state = State(tourists, set(), True)
    
    
    counter = 0
    priority_queue = [(0, counter, initial_state)]
    counter += 1
    
    # Setting to keep track of visited states
    visited = set()
    states_visited = 0
    
    while priority_queue:
        # Gettting state with minimum cost
        _, _, current_state = heapq.heappop(priority_queue)
        states_visited += 1
        
        # Skipping if we've visited this state before
        if current_state in visited:
            continue
        
        # Marking the state as visited
        visited.add(current_state)
        
        # Checking if we've reached the goal
        if current_state.is_goal():
            return current_state, states_visited
        
        # Gettting successors and adding them to the priority queue
        for successor in get_successors(current_state, crossing_times):
            if successor not in visited:
                heapq.heappush(priority_queue, (successor.time, counter, successor))
                counter += 1
    
    # If no solution is found
    return None, states_visited


def main():
    # input
    try:
        times = list(map(int, input("Enter the crossing times of the four tourists: ").strip().split()))
        if len(times) != 4:
            print("Please enter exactly 4 crossing times.")
            return
        
        
        crossing_times = {i+1: time for i, time in enumerate(times)}
        
        
        solution, states_visited = uniform_cost_search(crossing_times)
        
        if solution:
            print("\nOptimal sequence of moves:")
            for i, move in enumerate(solution.moves, 1):
                print(f"{i}. {move}")
            print(f"Total time taken: {solution.time} minutes")
            print(f"Total states visited: {states_visited}")
        else:
            print("No solution found.")
    
    except ValueError:
        print("Please enter valid integer crossing times.")


if __name__ == "__main__":
    main()


Optimal sequence of moves:
1. (1,2) cross → Time: 2
2. (1) returns → Time: 3
3. (3,4) cross → Time: 13
4. (2) returns → Time: 15
5. (1,2) cross → Time: 17
Total time taken: 17 minutes
Total states visited: 43
